In [ ]:
! pip install -q transformers
! pip install -q datasets
! pip install -q huggingface_hub
! pip install -q sentencepiece
! pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json

from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, pipeline, TrainingArguments, Trainer
from tqdm import tqdm
from IPython.display import clear_output
import torch
from huggingface_hub import notebook_login
import evaluate
from google.colab import drive
import pandas as pd

In [ ]:
model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 512
doc_stride = 256
batch_size = 8
lr = 3e-5
epoch = 8

In [ ]:
MODEL_PATH = "./"

In [ ]:
org_dataset = load_dataset("Hamid-reza/small-persian-QA")
adv_dataset = load_dataset("jalalnb/back_translation_hy_on_small_persian_QA")
# adv_dataset = load_dataset("jalalnb/back_translation_en_on_small_persian_QA")
# adv_dataset = load_dataset("jalalnb/back_translation_fr_on_small_persian_QA")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1261 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/130 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Hamid-reza___parquet/Hamid-reza--small-persian-QA-0ef749d72b9649a4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/130 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/mohammadhossein___parquet/mohammadhossein--addany-dataset-d9f1e2d06d1b9be9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
org_train_dataset = org_dataset["train"]
org_val_dataset = org_dataset["validation"]

adv_train_dataset = adv_dataset["train"]
adv_val_dataset = adv_dataset["validation"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
org_tokenized_train = org_train_dataset.map(prepare_train_features, batched=True, remove_columns=org_train_dataset.column_names)
org_tokenized_val = org_val_dataset.map(prepare_train_features, batched=True, remove_columns=org_val_dataset.column_names)

adv_tokenized_train = adv_train_dataset.map(prepare_train_features, batched=True, remove_columns=adv_train_dataset.column_names)
adv_tokenized_val = adv_val_dataset.map(prepare_train_features, batched=True, remove_columns=adv_val_dataset.column_names)

Map:   0%|          | 0/1261 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized 

In [ ]:
args = TrainingArguments(
    MODEL_PATH,
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001,
    save_strategy="no") 

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=org_tokenized_train,
    eval_dataset=org_tokenized_val,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1261
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1264
  Number of trainable parameters = 162252290
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,2.994762
2,No log,2.940382
3,No log,3.524318
4,2.437800,4.032407
5,2.437800,4.609015
6,2.437800,4.771095
7,0.271800,4.834877
8,0.271800,4.994017


***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1264, training_loss=1.0847156379796281, metrics={'train_runtime': 624.9506, 'train_samples_per_second': 16.142, 'train_steps_per_second': 2.023, 'total_flos': 1441467544759860.0, 'train_loss': 1.0847156379796281, 'epoch': 8.0})

In [ ]:
adv_trainer = Trainer(
    model=model,
    args=args,
    train_dataset=adv_tokenized_train,
    eval_dataset=adv_tokenized_val,
    tokenizer=tokenizer)

In [ ]:
adv_trainer.train()

***** Running training *****
  Num examples = 1000
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
  Number of trainable parameters = 162252290


Epoch,Training Loss,Validation Loss
1,No log,5.535750
2,No log,5.687025
3,No log,5.332330
4,0.127200,6.159936
5,0.127200,6.085704
6,0.127200,5.527086
7,0.127200,6.129885
8,0.018800,5.867611


***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1000, training_loss=0.07302416896820069, metrics={'train_runtime': 496.2886, 'train_samples_per_second': 16.12, 'train_steps_per_second': 2.015, 'total_flos': 1140878798574912.0, 'train_loss': 0.07302416896820069, 'epoch': 8.0})

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("bt_hy_parsbert_v1")
tokenizer.push_to_hub("bt_hy_parsbert_tokenizer_v1")

# model.push_to_hub("bt_en_parsbert_v1")
# tokenizer.push_to_hub("bt_en_parsbert_tokenizer_v1")

# model.push_to_hub("bt_fr_parsbert_v1")
# tokenizer.push_to_hub("bt_fr_parsbert_tokenizer_v1")

Configuration saved in /tmp/tmphypc5ozl/config.json
Model weights saved in /tmp/tmphypc5ozl/pytorch_model.bin
Uploading the following files to arashmarioriyad/addany_parsbert_v1: pytorch_model.bin,config.json


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/649M [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmpfhnrhc6j/tokenizer_config.json
Special tokens file saved in /tmp/tmpfhnrhc6j/special_tokens_map.json
Uploading the following files to arashmarioriyad/addany_parsbert_tokenizer_v1: special_tokens_map.json,vocab.txt,tokenizer.json,tokenizer_config.json


CommitInfo(commit_url='https://huggingface.co/arashmarioriyad/addany_parsbert_tokenizer_v1/commit/76b545d218e9be3ba1c9dee03e2dbc2117841326', commit_message='Upload tokenizer', commit_description='', oid='76b545d218e9be3ba1c9dee03e2dbc2117841326', pr_url=None, pr_revision=None, pr_num=None)